In [50]:
import torch
x = torch.tensor([
    [ 0.0470, -0.0431,  0.0421],
    [ 0.0282, -0.0119,  0.0402],
    [0.0181, 0.0000, 0.0000],
    [-0.0017,  0.0001, -0.0004],
    [ 4.1447e-03, -2.7173e-03, -2.3842e-07]
    ])
x.softmax(1)

tensor([[0.3438, 0.3141, 0.3421],
        [0.3364, 0.3232, 0.3404],
        [0.3374, 0.3313, 0.3313],
        [0.3330, 0.3336, 0.3334],
        [0.3346, 0.3323, 0.3332]])

In [81]:
torch.softmax(-x*20, 1)

tensor([[0.1225, 0.7424, 0.1351],
        [0.2490, 0.5552, 0.1958],
        [0.2582, 0.3709, 0.3709],
        [0.3403, 0.3282, 0.3315],
        [0.3093, 0.3548, 0.3360]])

In [71]:
torch.softmax(x.exp()-1, 1)

tensor([[0.3438, 0.3141, 0.3421],
        [0.3364, 0.3230, 0.3406],
        [0.3374, 0.3313, 0.3313],
        [0.3330, 0.3336, 0.3334],
        [0.3346, 0.3323, 0.3332]])

In [70]:
x.exp()-1

tensor([[ 4.8122e-02, -4.2184e-02,  4.2999e-02],
        [ 2.8601e-02, -1.1829e-02,  4.1019e-02],
        [ 1.8265e-02,  0.0000e+00,  0.0000e+00],
        [-1.6986e-03,  1.0002e-04, -3.9995e-04],
        [ 4.1533e-03, -2.7136e-03, -2.3842e-07]])

In [54]:
x.exp()/2

tensor([[0.5241, 0.4789, 0.5215],
        [0.5143, 0.4941, 0.5205],
        [0.5091, 0.5000, 0.5000],
        [0.4992, 0.5001, 0.4998],
        [0.5021, 0.4986, 0.5000]])

In [72]:
x1 = x - x.min(1, keepdim=True)[0] + 1e-3
x1 / x1.sum(1, keepdim=True)

tensor([[0.5109, 0.0056, 0.4835],
        [0.4317, 0.0105, 0.5578],
        [0.9052, 0.0474, 0.0474],
        [0.1639, 0.4590, 0.3770],
        [0.6250, 0.0795, 0.2955]])

## Download model

In [84]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "NousResearch/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model-00001-of-00004.safetensors:  29%|##9       | 1.46G/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]